In [1]:
(import hy sys)
(print "Hy version: " hy.__version__)
(print "Python" sys.version)

Hy version:  0.13.0
Python 3.6.2 | packaged by conda-forge | (default, Jul 23 2017, 22:58:45) [MSC v.1900 64 bit (AMD64)]


In [2]:
(require (calchylus.lambdas [*]))

In [3]:
(with-alpha-conversion-nor-macros L , )
; not lambda expression
(assert (= (, 'x) (, 'x)))
; no body lambda expression
(assert (= (L ,) None))
; constant
(assert (= (L , c) 'c))
; identity, without argument
(assert (= (L x , x) "(L x , x)"))
; identity, with argument
(assert (= (L x , x y) 'y))
; select first, without arguments
(assert (= (L x y , x) "(L x y , x)"))
; select second, without arguments
(assert (= (L x y , y) "(L x y , y)"))
; select first, with arguments
(assert (= (L x y , x 1 0) 1))
; select second, with arguments
(assert (= (L x y , y 1 0) 0))
; multiary function without arguments
(assert (= (L x y z , (x y z)) "(L x y z , (x y z))"))
; nested functions and arguments
(assert (= (L x , (L y , (L z , (z x y) l) k) j) "(l j k)"))
; nested functions, flatten arguments
(assert (= (L x , (L y , (L z , (z x y))) j k l) "(l j k)"))
; multiary function with arguments
(assert (= (L x y z , (z x y) j k l) "(l j k)"))
; application without arguments
(assert (= (L , ((L x y z , (x y z)))) "(L x y z , (x y z))"))
; application with arguments
(assert (= (L , ((L x y z , (x y z)) a b c)) "(a b c)"))
; any free arguments should be returned on the final result
(assert (= (L x , x y z) "(y z)"))
; nested free arguments
(assert (= (L , ((L x y z , (x y z) l m n) a b c) 1 2 3) "((l m n) a b c 1 2 3)"))

<function <lambda> at 0x00000276EB570BF8>

In [4]:
(with-alpha-conversion-nor-macros 𝜆 ·)
; using application shapr macro, starting with lambda expression
(assert (= #Ÿ(𝜆 x · x) "(𝜆 x · x)"))
; using application shapr macro, lambda expressionin the middle
(assert (= #Ÿ(x (x (𝜆 x · x y))) "(x (x y))"))
; return whole abstraction
(assert (= (𝜆 x · (𝜆 y · (𝜆 z · (x y z)))) "(𝜆 x · (𝜆 y · (𝜆 z · (x y z))))"))
; evaluate first abstraction, return rest abstraction levels
(assert (= (𝜆 x · (𝜆 y · (𝜆 z · (x y z))) 1) "(𝜆 y · (𝜆 z · (1 y z)))"))
; evaluate first and second abstraction, return rest abstraction levels
(assert (= (𝜆 x · (𝜆 y · (𝜆 z · (x y z))) 1 2) "(𝜆 z · (1 2 z))"))
; evaluate first, second, and the third abstraction, return fully evaluated result
(assert (= (𝜆 x · (𝜆 y · (𝜆 z · (x y z))) 1 2 3) "(1 2 3)"))

<function <lambda> at 0x00000276EB570D08>

In [5]:
(with-alpha-conversion-nor-macros L ,)
; without variable renaming next result is given
; first substitute x with y, and then both y's with z, resulting (z z)
(assert (= (L x y , (x y) y z) "(z z)"))
; same should apply to deeper values too
(assert (= (L x y , (x y) (y (y z)) z) "((z (z z)) z)"))

(with-alpha-conversion L , )
; using variable renaming / alpha conversion to prevent name collision should give right result
; instead of first substituting x with y, and then both y's with z
; this test should give (y z) because y is bound to x only. latter y should not replace it
(assert (= (L x y , (x y) y z) "(y z)"))
; same should apply to deeper values too. note similarity with LET and LET* macro handler!
(assert (= (L x y , (x y) (y (y z)) z) "((y (y z)) z)"))

<function <lambda> at 0x00000276EB3AB048>

In [6]:
(with-macros L , )
; special macros for main lambda terms / forms
; constant
(assert (= (CONST x) None))
(assert (= (CONST x 1) "(L x , 1)"))
(assert (= (CONST x 1 2) 1))
; identity, without arguments
(assert (= (IDENT) "(L a , a)"))
; identity, with arguments
(assert (= (IDENT 1) 1))
; identity, nested
(assert (= (IDENT (IDENT 1)) 1))
; boolean macros
(assert (= (, (TRUE) (TRUE 1 0) 
              (FALSE) (FALSE 1 0)) 
           (, "(L a b , a)" 1 
              "(L a b , b)" 0)))

<function <lambda> at 0x00000276EB56A0D0>

In [7]:
; lists
(assert (= (NIL) "(L a b , b)"))
; pair constructor
(assert (= (PAIR TRUE NIL) "(s (L a b , a) (L a b , b))"))
; selector
(assert (= (PAIR TRUE NIL TRUE) (TRUE)))
; selector
(assert (= (PAIR TRUE NIL FALSE) (NIL)))
; head and tail
(assert (= (, (HEAD (PAIR TRUE NIL)) 
              (TAIL (PAIR TRUE NIL))) 
           (, (TRUE) (NIL))))
; is nil, simple
(assert (= (, (NIL? NIL) (NIL? FALSE) (NIL? TRUE) (NIL? ONE))
           (, (TRUE) (TRUE) "(L a , (L a b , b))" (FALSE))))
; is nil, head and tail
(assert (= (, (NIL? (HEAD (PAIR ONE NIL))) 
              (NIL? (TAIL (PAIR ONE NIL)))) 
           (, (FALSE) (TRUE))))
; nested pairs and heads and tails
(assert (=
   (, (HEAD (PAIR TRUE (PAIR TRUE NIL))) 
      (HEAD (TAIL (PAIR TRUE (PAIR TRUE NIL))))
      (TAIL (TAIL (PAIR TRUE (PAIR TRUE NIL)))))
   (, (TRUE) (TRUE) (NIL))))
; nil? for nested pairs
(assert (=
  (, (NIL? (TAIL (TAIL (PAIR ONE (PAIR ONE NIL))))) 
     (NIL? (HEAD (TAIL (PAIR ONE (PAIR ONE NIL))))))
  (, (TRUE) (FALSE))))
; simple condition
(assert (=
    (, (COND TRUE TRUE FALSE) 
       (COND FALSE TRUE FALSE))
    (, (TRUE) (FALSE))))
; number nil? conditions
(assert (=
    (, (COND (NIL? (NUM 0)) TRUE FALSE)
       (COND (NIL? (NUM 1)) TRUE FALSE)
       (COND (NIL? (NUM 10)) TRUE FALSE))
    (, (TRUE) (FALSE) (FALSE))))
; nil tail/head pair condition
(assert (=
    (, (COND (NIL? (TAIL (PAIR (NUM 1) NIL))) TRUE FALSE)
       (COND (NIL? (HEAD (PAIR (NUM 1) NIL))) TRUE FALSE))
    (, (TRUE) (FALSE))))
; TODO: this requires some thinking, nil to lists should be done with
; empty? check?
(assert (= (, (NIL? (PAIR ONE TWO)) (NIL? (LIST ONE TWO)) (NIL? (LIST)))
           (, (TRUE) (TRUE) (TRUE))))

In [8]:
; church numeral general generator, without arguments
(assert (= (NUM 3) "(L x y , (x (x (x y))))"))
; church numeral, general generator, with arguments
(assert (= (NUM 3 m n) "(m (m (m n)))"))
; church numeral, zero
(assert (= (ZERO) "(L a b , b)"))
; church numeral, zero with arguments
(assert (= (ZERO a b) 'b))
(assert (= (, (ZERO) (ZERO x y) (ONE) (TWO x y) (THREE m n))
           (, "(L a b , b)" "y" "(L x y , (x y))" "(x (x y))" "(m (m (m n)))")))
; if TRUE is given, then the third option is activated!
(assert (= (ZERO? TRUE a b c) 'c))

In [9]:
; lists
; empty
(assert (= (LIST) 
           "(s (L a b , b) (L a b , b))"))
(assert (= (LIST NIL) 
           "(s (L a b , b) (s (L a b , b) (L a b , b)))"))
; one item
(assert (= (LIST ONE) 
           "(s (L x y , (x y)) (s (L a b , b) (L a b , b)))"))
; three items
(assert (= (LIST ONE TWO) 
           "(s (L x y , (x y)) (s (L x y , (x (x y))) (s (L a b , b) (L a b , b))))"))
; first
(assert (= (FIRST (LIST ONE TWO THREE)) (ONE)))
; second
(assert (= (SECOND (LIST ONE TWO THREE)) (TWO)))
; last
(assert (= (LAST (LIST ONE TWO THREE)) (THREE)))
; append to the end of the list
(assert (= (LAST (APPEND FOUR (LIST ONE TWO THREE))) (FOUR)))

In [10]:
; letters
(assert (= (LET a 1 b 2 d 3 (a b c d)) "(1 2 c 3)"))
(assert (= (LET a 1 (a a (L a , a 2))) "(1 1 2)"))
(assert (= (LET a 1 b a (a b)) "(1 a)"))
(assert (= (LET a 1 c (a b) (LET b a c)) "(a 1)"))
(assert (= (LET a 1 c (LET b a c) (a b)) "(1 b)"))
(assert (= (LET a 1 b (a b) b) "(a b)"))
(assert (= (LET a 1 (LET b a (a b))) "(1 1)"))
(assert (= (LET* a 1 b a (a b)) "(1 1)"))
(assert (= (LET x) "x"))
(assert (= (LET* x) "x"))
(assert (= (LET) "()"))
(assert (= (LET*) "()"))

In [11]:
; do structure
(assert (= (DO (LET a 1) (LET b 2) (LET c b) (a b c d))  "(1 2 2 d)"))
(assert (L , (COND (DO (TRUE) (FALSE) (TRUE) (FALSE)) TRUE FALSE) "(L a b , b)"))
(assert (L , (COND (DO (TRUE) (TRUE) (TRUE) (TRUE)) TRUE FALSE) "(L a b , a)"))

In [12]:
; and tests
(assert (=
  (, (AND TRUE TRUE) (AND TRUE FALSE)
     (AND FALSE TRUE) (AND FALSE FALSE))
  (, (TRUE) (FALSE) (FALSE) (FALSE))))
; or tests
(assert (=
  (, (OR TRUE TRUE) (OR TRUE FALSE)
     (OR FALSE TRUE) (OR FALSE FALSE))
  (, (TRUE) (TRUE) (TRUE) (FALSE))))
; not tests
(assert (= (, (NOT TRUE) (NOT FALSE)) (, (FALSE) (TRUE))))
; xor tests
(assert (= 
  (, (XOR TRUE TRUE) (XOR TRUE FALSE)
     (XOR FALSE TRUE) (XOR FALSE FALSE))
  (, (FALSE) (TRUE) (TRUE) (FALSE))))
; logic condition
(assert (= (COND (AND (NOT (XOR FALSE FALSE)) (OR TRUE FALSE)) TRUE FALSE) (TRUE)))
; eval read str
(assert (= (eval (read-str "(TRUE)")) "(L a b , a)"))
; lesser or equal
(assert (= (, (LEQ? ONE TWO) (LEQ? TWO ONE)  (LEQ? ONE ONE) 
           (, (TRUE) (FALSE) (TRUE)))))
; greater or equal
(assert (= (, (GEQ? ONE TWO) (GEQ? TWO ONE)  (GEQ? ONE ONE) 
           (, (FALSE) (TRUE) (TRUE)))))
; equals
(assert (= (, (EQ? ONE TWO) (EQ? TWO ONE)  (EQ? ONE ONE) 
           (, (FALSE) (FALSE) (TRUE)))))
; lesser
(assert (= (, (LE? ONE TWO) (LE? TWO ONE)  (LE? ONE ONE) 
           (, (TRUE) (FALSE) (FALSE)))))
; greater
(assert (= (, (GE? ONE TWO) (GE? TWO ONE)  (GE? ONE ONE) 
           (, (FALSE) (TRUE) (FALSE)))))

In [13]:
; math operations
; next, inc, successive
(assert (= (SUCC ONE) "(x (x y))"))
; infix notation
(assert (= (ONE SUCC ONE SUCC ONE) "(x (x (x y)))"))
; previous, dec, predecessive
(assert (= (PRED THREE) "(x (x y))"))
; previous, dec, predecessor, with arguments
(assert (= (PRED THREE a b) "(a (a b))"))
; nested predecessor
(assert (= (PRED (PRED (PRED FOUR))) "(x y)"))
; previous + next is same
(assert (= (SUCC (PRED TWO)) "(x (x y))"))
; previous + next is same for zero
(assert (= (PRED (SUCC ZERO)) 'y))
; but previous + next is one for zero!
(assert (= (SUCC (PRED ZERO)) "(x y)"))
; sum two values
(assert (= (SUM TWO TWO) "(x (x (x (x y))))"))
; substract two x from y
(assert (= (, (SUB ONE TWO) (SUB ONE ONE) (SUB TWO ONE))
           (, "(x y)" 'y 'y)))
(assert (= (, (EXP TWO TWO) (EXP TEN ZERO) (EXP ZERO ZERO))
           (, "(x (x (x (x y))))" "(x y)" "(x y)")))
(assert (= (, (PROD ZERO ONE a b) (PROD ONE ONE a b) (PROD TWO TWO a b))
           (, 'b "(a b)" "(a (a (a (a b))))")))

In [14]:
; self application
(assert (= (SELF (L x , x) 1) 1))
; self application, fixed point
;count down to zero
(assert (= (SELF (L f n , (COND (ZERO? n) ZERO (f f (PRED n)))) THREE) (ZERO)))
; count down to one with lesser or equal comparison
(assert (= (SELF (L f n , (COND (LEQ? n ONE) n (f f (PRED n)))) FOUR) "(x y)"))
; count down to one with equal comparison
(assert (= (SELF (L f n , (COND (EQ? n ONE) n (f f (PRED n)))) FOUR) "(x y)"))

In [15]:
; summation sequence, with plain number
(assert (= (SUMMATION (L x y , (x (x (x y))))) "(x (x (x (x (x (x y))))))"))
; summation sequence, with number macro form
(assert (= (SUMMATION THREE) "(x (x (x (x (x (x y))))))"))
; product sequence, with plain number
(assert (= (FACTORIAL (L x y , (x (x (x y))))) "(x (x (x (x (x (x y))))))"))
; product sequence, with number macro form
(assert (= (FACTORIAL THREE) "(x (x (x (x (x (x y))))))"))
; fibonacci, with plain number
(assert (= (FIBONACCI (L x y , (x (x (x (x (x y))))))) "(x (x (x (x (x y)))))"))
; fibonacci, with number macro form
(assert (= (FIBONACCI FIVE) "(x (x (x (x (x y)))))"))

In [16]:
; self application recursive loop
(setv result (L x , (x x) (L x , (x x))))
(assert (none? result))

Recursion error occured for lambda expression:  (x , (x x) (L x , (x x)))


In [17]:
(SUMMATION (NUM 8))

'(x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x y))))))))))))))))))))))))))))))))))))'

In [18]:
(FIBONACCI (NUM 7))

'(x (x (x (x (x (x (x (x (x (x (x (x (x y)))))))))))))'

In [19]:
; requires alpha conversion and macros, or will cause recursion error!
(with-alpha-conversion-and-macros L , )
(FACTORIAL (NUM 5))

'(x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x (x y))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))'

## The [MIT](http://choosealicense.com/licenses/mit/) License

Copyright (c) 2017 Marko Manninen